## Exercise 2
In the course you learned how to do classificaiton using Fashion MNIST, a data set containing items of clothing. There's another, similar dataset called MNIST which has items of handwriting -- the digits 0 through 9.

Write an MNIST classifier that trains to 99% accuracy or above, and does it without a fixed number of epochs -- i.e. you should stop training once you reach that level of accuracy.

Some notes:
1. It should succeed in less than 10 epochs, so it is okay to change epochs= to 10, but nothing larger
2. When it reaches 99% or greater it should print out the string "Reached 99% accuracy so cancelling training!"
3. If you add any additional variables, make sure you use the same names as the ones used in the class

The following contains my implementation

In [1]:
import numpy as np
np.set_printoptions(linewidth=200)
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
import tensorflow.keras.backend as K

In [3]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        thres = 1
        if(logs.get('accuracy') is not None and logs.get('accuracy') >= thres) : # Experiment with changing this value
            print(f"\nReached {thres*100}% accuracy on epoch = {epoch+1}!")
            self.model.stop_training = True

In [4]:
# GRADED FUNCTION: train_mnist
def train_cnn_mnist():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    callback =myCallback()
    num_classes = 10
    #as we have 10 classes (0-9) 
    #we need to prdeict one out of 10 which has high probability
    epochs = 30
    img_rows = 28 
    img_cols = 28
    #as each image is 28 by 28 pixel

    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    # YOUR CODE SHOULD START HERE
    x_train =x_train.astype(float)
    x_train =x_train/255
    x_test =x_test.astype(float)
    x_test =x_test/255

    y_train = to_categorical(y_train, num_classes = 10, dtype = 'float32')
    y_test = to_categorical(y_test, num_classes = 10, dtype = 'float32')

    if K.image_data_format() =='channels_first':
        x_train = x_train.reshape(x_train.shape[0],1,img_rows,img_cols)
        x_test = x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
        input_shape = (1,img_rows,img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
        x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
        input_shape = (img_rows,img_cols,1)
    # YOUR CODE SHOULD END HERE
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # model fitting
    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[callback], verbose=1)

    # saving model
    model.save("./artifacts/cnn_model.h5")
    # model fitting
    return history.epoch, history.history['accuracy'][-1]

In [5]:
train_cnn_mnist()

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 33s 11ms/step - loss: 0.3092 - accuracy: 0.9116 - val_loss: 0.0624 - val_accuracy: 0.9798
Epoch 2/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.1025 - accuracy: 0.9726 - val_loss: 0.0520 - val_accuracy: 0.9844
Epoch 3/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0806 - accuracy: 0.9784 - val_loss: 0.0489 - val_accuracy: 0.9851
Epoch 4/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0696 - accuracy: 0.9818 - val_loss: 0.0408 - val_accuracy: 0.9877
Epoch 5/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0584 - accuracy: 0.9843 - val_loss: 0.0304 - val_accuracy: 0.9908
Epoch 6/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0511 - accuracy: 0.9867 - val_loss: 0.0354 - val_accuracy: 0.9899
Epoch 7/10
1875/1875 [==============================] - 20s

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 0.9902833104133606)

# 

In [1]:
!tensorflowjs_converter --input_format keras ./artifacts/cnn_model.h5 ./artifacts/cnn_model